# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import ipywidgets as widgets
from IPython.display import display
from numpy import trapz

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
#import eegyolk
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata.head()

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,105a,105,a,f,17,f,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,105a_epo.fif
1,107a,107,a,f,16,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,107a_epo.fif
2,106a,106,a,m,19,f,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,106a_epo.fif
3,109a,109,a,m,21,m,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,109a_epo.fif
4,110a,110,a,m,17,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,110a_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.2 to 0.8.

In [4]:
epochs = initialization_functions.read_filtered_data(metadata)

Checking out file: 105a_epo.fif
Checking out file: 107a_epo.fif
Checking out file: 106a_epo.fif
Checking out file: 109a_epo.fif
Checking out file: 110a_epo.fif
Checking out file: 112a_epo.fif
Checking out file: 111a_epo.fif
Checking out file: 114a_epo.fif
Checking out file: 115a_epo.fif
Checking out file: 117a_epo.fif
Checking out file: 116a_epo.fif
Checking out file: 118a_epo.fif
Checking out file: 119a_epo.fif
Checking out file: 123a_epo.fif
Checking out file: 122a_epo.fif
Checking out file: 124a_epo.fif
Checking out file: 127a_epo.fif
Checking out file: 125a_epo.fif
Checking out file: 126a_epo.fif
Checking out file: 130a_epo.fif
Checking out file: 128a_epo.fif
Checking out file: 129a_epo.fif
Checking out file: 131a_epo.fif
Checking out file: 135a_epo.fif
Checking out file: 133a_epo.fif
Checking out file: 137a_epo.fif
Checking out file: 138a_epo.fif
Checking out file: 139a_epo.fif
Checking out file: 141a_epo.fif
Checking out file: 144a_epo.fif
Checking out file: 143a_epo.fif
Checking

In [5]:
len(epochs)

101

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [12]:
def input_mmr_prep(metadata, epochs, standard_events, deviant_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file",  "channel", 'std', 'sur', 'min', 'max']) # "paradigm",

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        
        std_evoked = epochs[i][standard_events].average() 
        dev_evoked = epochs[i][deviant_events].average()
        
        chnames_list = [ 'FC5', 'Pz', 'O1', 'PO4', 'AF4']
        
        for channel in chnames_list: 
            evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1]).get_data(picks=channel) # calculate the mismatch response between standard and deviant evoked
            evoked_diff = np.reshape(evoked_diff, 2049)

            #chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            #roi_dict = dict(left_ROI=chnames) # combine_channels only takes a dictionary as input
            #roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            #mmr = roi_evoked.to_data_frame()
            #mmr_avg = evoked_diff.mean()
            mmr_std = evoked_diff.std()
            area = trapz(evoked_diff)
            mmr_sur = area
            
            zerocross= 0
            #for j in range(1, len(evoked_diff)): 
            #    
            #    if ((evoked_diff[j-1]) > 0 and evoked_diff[j] < 0):
            #        zerocross +=1
            #    if ((evoked_diff[j-1]) < 0 and evoked_diff[j] > 0):
            #        zerocross +=1
            #                   
            #mmr_zero = zerocross
            #mmr_skew = evoked_diff.skew()
            #mmr_var = mmr['left_ROI'].var()
            #mmr_kurt = evoked_diff.kurtosis()
            mmr_min = evoked_diff.min()
            mmr_max = evoked_diff.max()
            #df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var,'min' : mmr_min,'max' : mmr_max}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms 
   
            df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'std' : mmr_std, 'sur' : mmr_sur, 'min' : mmr_min, 'max' : mmr_max}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms 
    return df

In [13]:
# define the events for standard and deviant
standard_events = ['GiepM_S','GiepS_S','GopM_S','GopS_S'] #'GiepM_S','GiepS_S','GopM_S','GopS_S'
deviant_events = ['GiepM_D','GiepS_D','GopM_D','GopS_D'] #'GiepM_D','GiepS_D','GopM_D','GopS_D'


df = input_mmr_prep(metadata, epochs, standard_events, deviant_events)

/tmp/ipykernel_1331760/803427575.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'std' : mmr_std, 'sur' : mmr_sur, 'min' : mmr_min, 'max' : mmr_max}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms
/tmp/ipykernel_1331760/803427575.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'std' : mmr_std, 'sur' : mmr_sur, 'min' : mmr_min, 'max' : mmr_max}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms
/tmp/ipykernel_1331760/803427575.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'eeg_fi

In [14]:
df

,eeg_file,channel,std,sur,min,max
0,105a,FC5,2.365847e-06,0.004263,-0.000002,0.000009
1,105a,Pz,1.060541e-06,0.001750,-0.000001,0.000003
2,105a,O1,2.899746e-06,0.008843,-0.000003,0.000009
3,105a,PO4,1.299175e-06,-0.001303,-0.000004,0.000002
4,105a,AF4,3.685909e-06,0.006662,-0.000001,0.000012
...,...,...,...,...,...,...
500,221a,FC5,9.421346e-07,0.000051,-0.000002,0.000002
501,221a,Pz,9.055398e-07,-0.000390,-0.000002,0.000002
502,221a,O1,1.022857e-06,-0.003402,-0.000003,0.000001
503,221a,PO4,9.280260e-07,0.000167,-0.000002,0.000002


In [15]:
df = df.drop_duplicates(subset=['eeg_file','channel']) # ,'paradigm'

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [16]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['channel']) # 'paradigm',

In [17]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]

In [18]:
df.reset_index(inplace=True)

## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [19]:
df = pd.merge(df, metadata, on='eeg_file')

In [20]:
pd.set_option('display.max_columns', None)

In [21]:
df

,eeg_file,std_AF4,std_FC5,std_O1,std_PO4,std_Pz,sur_AF4,sur_FC5,sur_O1,sur_PO4,sur_Pz,min_AF4,min_FC5,min_O1,min_PO4,min_Pz,max_AF4,max_FC5,max_O1,max_PO4,max_Pz,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,1.384333e-06,1.019953e-06,0.000002,1.925103e-06,1.365521e-06,-0.002290,-0.001389,-0.001222,-0.004980,-0.003422,-0.000005,-0.000004,-0.000004,-0.000006,-4.576352e-06,0.000002,0.000001,0.000003,0.000001,9.479040e-07,101,a,m,20,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,101a_epo.fif
1,103a,1.525937e-06,2.198310e-06,0.000016,3.247296e-06,3.280949e-06,-0.000986,-0.003107,0.033049,0.007922,0.008508,-0.000004,-0.000007,-0.000010,-0.000002,-2.467039e-06,0.000003,0.000003,0.000043,0.000010,9.259968e-06,103,a,f,20,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,103a_epo.fif
2,104a,1.344636e-06,7.629572e-07,0.000001,1.635251e-06,1.049152e-06,-0.001301,-0.000635,0.001034,0.000391,-0.000471,-0.000004,-0.000002,-0.000002,-0.000003,-3.084455e-06,0.000002,0.000002,0.000004,0.000004,2.285211e-06,104,a,m,18,f,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,104a_epo.fif
3,105a,3.685909e-06,2.365847e-06,0.000003,1.299175e-06,1.060541e-06,0.006662,0.004263,0.008843,-0.001303,0.001750,-0.000001,-0.000002,-0.000003,-0.000004,-1.359372e-06,0.000012,0.000009,0.000009,0.000002,2.689267e-06,105,a,f,17,f,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,105a_epo.fif
4,106a,1.523331e-06,1.672483e-06,0.000002,2.113902e-06,1.382373e-06,-0.000429,-0.003228,0.000640,0.001983,0.001934,-0.000004,-0.000005,-0.000003,-0.000002,-1.412269e-06,0.000003,0.000002,0.000006,0.000007,4.757814e-06,106,a,m,19,f,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,106a_epo.fif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,217a,1.168110e-06,1.105110e-06,0.000002,1.851433e-06,8.426879e-07,-0.000892,0.001617,0.002010,0.005189,0.000383,-0.000003,-0.000001,-0.000002,-0.000002,-2.142243e-06,0.000003,0.000003,0.000006,0.000005,2.009517e-06,217,a,f,18,f,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,217a_epo.fif
97,218a,1.354678e-06,1.530969e-06,0.000001,1.151433e-06,8.372269e-07,0.000771,0.000575,-0.000865,-0.002496,0.000467,-0.000003,-0.000002,-0.000003,-0.000004,-1.512354e-06,0.000003,0.000004,0.000003,0.000001,1.957813e-06,218,a,m,18,Nee,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,218a_epo.fif
98,219a,1.533014e-06,1.305128e-06,0.000002,1.746317e-06,1.377061e-06,0.003447,0.001934,0.006789,0.004278,0.004397,-0.000002,-0.000001,-0.000002,-0.000001,-7.816863e-07,0.000005,0.000004,0.000008,0.000005,4.103796e-06,219,a,m,18,m,1,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,219a_epo.fif
99,220a,8.269482e-07,8.132629e-07,0.000002,1.702823e-06,1.294922e-06,-0.000415,0.001225,-0.002209,-0.004435,-0.004214,-0.000003,-0.000001,-0.000006,-0.000005,-4.109057e-06,0.000002,0.000003,0.000003,0.000002,9.729952e-07,220,a,m,19,Nee,0,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,220a_epo.fif


Drop some unnecessary columns. 

In [22]:
df = df.drop(['eeg_file',
       'dyslexic_parent', 'path_eeg','path_epoch',
       'epoch_file', 'path_eventmarkers'], axis =1)

In [23]:
df['sex'] = np.where(
    (df['sex']=='m'), 1,0)

In [24]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

In [25]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

In [26]:
df

,Group_AccToParents,std_AF4,std_FC5,std_O1,std_PO4,std_Pz,sur_AF4,sur_FC5,sur_O1,sur_PO4,sur_Pz,min_AF4,min_FC5,min_O1,min_PO4,min_Pz,max_AF4,max_FC5,max_O1,max_PO4,max_Pz,ParticipantID,test,sex,age_months
0,1,1.384333e-06,1.019953e-06,0.000002,1.925103e-06,1.365521e-06,-0.002290,-0.001389,-0.001222,-0.004980,-0.003422,-0.000005,-0.000004,-0.000004,-0.000006,-4.576352e-06,0.000002,0.000001,0.000003,0.000001,9.479040e-07,101,a,1,20
1,1,1.525937e-06,2.198310e-06,0.000016,3.247296e-06,3.280949e-06,-0.000986,-0.003107,0.033049,0.007922,0.008508,-0.000004,-0.000007,-0.000010,-0.000002,-2.467039e-06,0.000003,0.000003,0.000043,0.000010,9.259968e-06,103,a,0,20
2,1,1.344636e-06,7.629572e-07,0.000001,1.635251e-06,1.049152e-06,-0.001301,-0.000635,0.001034,0.000391,-0.000471,-0.000004,-0.000002,-0.000002,-0.000003,-3.084455e-06,0.000002,0.000002,0.000004,0.000004,2.285211e-06,104,a,1,18
3,1,3.685909e-06,2.365847e-06,0.000003,1.299175e-06,1.060541e-06,0.006662,0.004263,0.008843,-0.001303,0.001750,-0.000001,-0.000002,-0.000003,-0.000004,-1.359372e-06,0.000012,0.000009,0.000009,0.000002,2.689267e-06,105,a,0,17
4,0,1.523331e-06,1.672483e-06,0.000002,2.113902e-06,1.382373e-06,-0.000429,-0.003228,0.000640,0.001983,0.001934,-0.000004,-0.000005,-0.000003,-0.000002,-1.412269e-06,0.000003,0.000002,0.000006,0.000007,4.757814e-06,106,a,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1.168110e-06,1.105110e-06,0.000002,1.851433e-06,8.426879e-07,-0.000892,0.001617,0.002010,0.005189,0.000383,-0.000003,-0.000001,-0.000002,-0.000002,-2.142243e-06,0.000003,0.000003,0.000006,0.000005,2.009517e-06,217,a,0,18
97,0,1.354678e-06,1.530969e-06,0.000001,1.151433e-06,8.372269e-07,0.000771,0.000575,-0.000865,-0.002496,0.000467,-0.000003,-0.000002,-0.000003,-0.000004,-1.512354e-06,0.000003,0.000004,0.000003,0.000001,1.957813e-06,218,a,1,18
98,1,1.533014e-06,1.305128e-06,0.000002,1.746317e-06,1.377061e-06,0.003447,0.001934,0.006789,0.004278,0.004397,-0.000002,-0.000001,-0.000002,-0.000001,-7.816863e-07,0.000005,0.000004,0.000008,0.000005,4.103796e-06,219,a,1,18
99,0,8.269482e-07,8.132629e-07,0.000002,1.702823e-06,1.294922e-06,-0.000415,0.001225,-0.002209,-0.004435,-0.004214,-0.000003,-0.000001,-0.000006,-0.000005,-4.109057e-06,0.000002,0.000003,0.000003,0.000002,9.729952e-07,220,a,1,19
